In [10]:
import numpy as np
import pandas as pd
import math

import src.hdmm.workload as workload
import src.census_workloads as census
from src.workload_selection import workload_selection
import online_workloads as online_workloads

# Database

In [55]:
data_path = "migration_tworace.csv"
x_data = pd.read_csv(data_path, header=None).to_numpy().T[1]
n = x_data.shape[0]
n

86

# Workloads

In [56]:
W_name = ['identity', 'total', 'race1', 'race2', 'race3', 'custom', 'prefix_sum']
W_lst = [online_workloads.identity(n), online_workloads.total(n), online_workloads.race1(), online_workloads.race2(), online_workloads.race3(), online_workloads.custom(n), online_workloads.prefix_sum(n)]

#online_workloads.H2(n), 'H2'

def print_workload(workload_name, workload):
    print(f'---Workload: {workload_name}---')
    print(f'Shape: {workload.shape}')
    print(f'Workload: \n{workload}\n')
    
def print_workloads():
    for i in range(7):
        print_workload(W_name[i], W_lst[i])
        
print_workloads()


---Workload: identity---
Shape: (86, 86)
Workload: 
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]

---Workload: total---
Shape: (1, 86)
Workload: 
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]

---Workload: race1---
Shape: (7, 64)
Workload: 
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.

In [17]:
len(online_workloads.identity(n))

86

In [57]:
x_data

array([412, 333, 285, 231, 202, 174, 160, 142, 146, 149, 145, 181, 174,
       190, 213, 287, 372, 499, 619, 715, 785, 821, 822, 816, 799, 742,
       717, 697, 658, 593, 564, 519, 447, 403, 388, 365, 336, 306, 311,
       289, 261, 231, 213, 196, 194, 170, 175, 168, 149, 142, 131, 119,
       112, 118, 114, 116, 112, 114, 106, 111, 109, 112, 113, 109, 104,
       108, 108,  94,  91,  81,  81,  72,  68,  63,  56,  46,  41,  38,
        34,  28,  23,  22,  18,  18,  16,  41])

In [73]:
# Todo: Add code to make epsilon adjustable proportionally. 
# Todo: Fix divide by zero error (<ipython-input-51-46063bd03d8a>:37: RuntimeWarning: divide by zero encountered in double_scalars delta = 1 / (n * math.log(n, np.e)))

def pmw_split(workload, x, analyst_labels, T=5, eps=0.01, k=5, show_messages=True, to_return='pd', show_plot=False, show_failure_step=True):
    """
    Implement Private Multiplicative Weights Mechanism (PMW) on a workload of
    linear queries. 

    Algorithm Parameters: 
    - workload = workload of queries (M x k numpy array)
    - x = true database (M x 1 numpy array)
    - T = update threshold
    - eps = privacy budget
    - k = number of update steps PER ANALYST
    - analyst_labels = list of analyst names corresponding to each query in the workload
    
    Output Controls: 
    - show_messages argument determines whether the function will print information such as 
    error scale, threshold, update steps used, etc.
    - to_return argument determines what the function will return. 
        - if 'pd', pmw() returns pandas df with test data for each 
        query in the workload(showing query, d_t_hat, updated, algo_ans, real_ans, 
        abs_error, rel_error). 
        - if 'update_count', pmw() returns the update count for the total
        amount of queries.
    - show_plot - T/F whether the function will display a plot
    - show_failure_step - T/F whether function prints what step failure mode is reached
    """ 
    
    update_steps = {}
    for analyst in list(set(analyst_labels)): 
        update_steps[analyst] = k # each analyst starts with k update steps
    
    # initialize constants
    m = x.size  # database len
    n = x.sum()
    eta = (math.log(m, np.e) ** (1 / 4)) / (math.sqrt(n))
    delta = 1 / (n * math.log(n, np.e))
    x_norm = x / np.sum(x)
    
    # initialize synthetic databases at time 0 (prior to any queries)
    x_t = np.ones(m) / m
    y_t = np.ones(m) / m

    # initialize tracker lists to construct pandas dataframe at the end 
    x_list = [x_t] # create a list of x_t synthetic database at every time step
    update_list = []
    update_count = 0
    pmw_answers = []
    update_times = [] # record times that database is updated
    d_t_hat_list = []
    
    def lazy_round():
        """
        "Lazy Round" of querying using the stored synthetic database, x_t, in list x_list.
        p
        We call this the lazy round because it is contrasted with the updated step where we update the 
        sythetic database and answer the query using the real database.
        """
        update_list.append('no')
        pmw_answers.append(np.dot(query, x_list[time]))
        x_list.append(x_list[time].round(3))
    
    # inititate first instance of SVT with half the budget and k updates; will be reset in the main loop
    SVTtrigger = False 
    SVTepsilon1 = ((eps/2)/2)
    SVTepsilon2 = ((eps/2)/2)
    rho = np.random.laplace(loc=0, scale=(1/SVTepsilon1), size=1)[0]
    
    for time, query in enumerate(workload):
        
        analyst = analyst_labels[time]
        
        # Do one round of sparse vector technique 
        
        # Compute noisy answer by adding Laplacian noise
        a_t = np.random.laplace(loc=0, scale=(2*k/SVTepsilon2), size=1)[0]
    
        a_t_hat = (np.dot(query, x_norm)*n ) + a_t
        
        # Difference between noisy and maintained histogram answer
        d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
        
        # Lazy round: use synthetic base to answer the query
        if (abs(d_t_hat) <= T + rho):
            d_t_hat_list.append(d_t_hat)
            lazy_round()
            continue

        # update round: update histogram and return noisy answer
        else:
            #make a new noisy query answer using some of the leftover budget
            a_t = np.random.laplace(loc=0, scale=(2*k/eps), size=1)[0]
            a_t_hat = (np.dot(query, x_norm)*n ) + a_t
            d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
            d_t_hat_list.append(d_t_hat)
            update_times.append(time)
            
            # step a
            if d_t_hat < 0:
                r_t = query
            else:
                r_t = np.ones(m) - query
            for i, v in enumerate(y_t):
                y_t[i] = x_list[time][i] * math.exp((d_t_hat/(2*n)) * query[i]) * 20 # 20 is the learning rate
            
            # step b
            x_t = y_t / np.sum(y_t)
            update_count = update_list.count('yes')
            
            # if threshold for num updates is reached, just do a lazy round (synthetic database) answer
            if update_steps[analyst] == 0: 
                if show_failure_step:
                    print(f'Failure mode reached at query number {time}: {query}')
                lazy_round()
                
            # if there are still update steps that the analyst can use, 
            # 1. update the synthetic database
            # 2. answer the query using the noisy answer from the database itself 
            else: 
                x_list.append(x_t.round(3))
                update_list.append('yes') # increment number of updates counter
                pmw_answers.append(a_t_hat / np.sum(x))
                update_steps[analyst] -= 1 # use one of analyst's update steps

    update_count = update_list.count('yes')      

    # calculate error
    real_ans = np.matmul(workload, x_norm)
    abs_error = np.abs(pmw_answers - real_ans)
    rel_error = np.abs(abs_error / np.where(real_ans == 0, 0.000001,
                                                real_ans))
    
    if show_messages:
        np.set_printoptions(suppress=True)
        """Print inputes/outputs to analyze each query"""
        print(f'Original database: {x}\n')
        print(f'Normalized database: {x_norm}\n')
        print(f'Updated Database = {x_t}\n')
        print(f'Update Count = {update_count}\n')
        print(f'{T=}\n')
        print(f'Error Scale Query Answer= {2*((2*k/eps)**2)}\n')
        print(f'Error Scale SVT= {2*((2*k/SVTepsilon2)**2)}\n')
        print(f'Update Parameter Scale = {eta}\n')
        print(f'{delta=}\n')
        
    if show_plot: 
        plt.title('Error across queries:')
        rel_line, = plt.plot(rel_error, label='Relative Error')
        abs_line, = plt.plot(abs_error, label='Absolute Error')
        for xc in update_times:
            plt.axvline(x=xc, color='red', label='Update Times', linestyle='dashed')
        plt.legend(handles=[abs_line,rel_line])
        plt.xticks(range(0, len(workload), round(len(workload)/5)))
    
    if to_return == "pd":
        # hacky fix: remove the first synthetic database to keep length of lists consistent with the
        # other lists that comprise of the pandas dataframe
        x_list.pop(len(x_list) - 1).tolist() 
        d = {
            'algo_ans': pmw_answers,
            'real_ans': real_ans.tolist(),
            'queries': workload.tolist(), 
            'updated': update_list,
            'abs_error': abs_error,               
            'rel_error': rel_error,
            'synthetic database': x_list,
            'analyst': analyst_labels,
            'd_t_hat': d_t_hat_list, 

             }
        test_data = pd.DataFrame(data=d)
        test_data = test_data.round(3)
        return test_data
    
    # return dictionary of absolute errors
    if to_return == "error":
        d = {'analyst': analyst_labels,
             'abs_error': abs_error,               
             'rel_error': rel_error,}
        data = pd.DataFrame(data=d)
        data = data.round(3)
        
        analyst_error = {}
        for analyst in list(set(analyst_labels)):
            analyst_error[analyst] = data[data.analyst==analyst]['abs_error'].sum()
        return analyst_error
    
x_example = np.array([1, 2, 3, 4, 5])
pmw_split(np.vstack((online_workloads.identity(5), online_workloads.identity(5))), x_example, ['Alice'] * 10, eps=10)



Original database: [1 2 3 4 5]

Normalized database: [0.06666667 0.13333333 0.2        0.26666667 0.33333333]

Updated Database = [0.19515278 0.19017439 0.19714413 0.20843643 0.20909226]

Update Count = 5

T=5

Error Scale Query Answer= 2.0

Error Scale SVT= 32.0

Update Parameter Scale = 0.29081910083756185

delta=0.024617958204590337



,algo_ans,real_ans,queries,updated,abs_error,rel_error,synthetic database,analyst,d_t_hat
0,0.200,0.067,"[1.0, 0.0, 0.0, 0.0, 0.0]",no,0.133,2.000,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,-4.413
1,0.200,0.133,"[0.0, 1.0, 0.0, 0.0, 0.0]",no,0.067,0.500,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,-3.817
2,0.200,0.200,"[0.0, 0.0, 1.0, 0.0, 0.0]",no,0.000,0.000,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,0.166
3,0.295,0.267,"[0.0, 0.0, 0.0, 1.0, 0.0]",yes,0.028,0.107,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,1.427
4,0.318,0.333,"[0.0, 0.0, 0.0, 0.0, 1.0]",yes,0.015,0.045,"[0.198, 0.198, 0.198, 0.208, 0.198]",Alice,1.804
5,0.177,0.067,"[1.0, 0.0, 0.0, 0.0, 0.0]",yes,0.110,1.652,"[0.196, 0.196, 0.196, 0.205, 0.208]",Alice,-0.288
6,0.126,0.133,"[0.0, 1.0, 0.0, 0.0, 0.0]",yes,0.008,0.057,"[0.194, 0.196, 0.196, 0.205, 0.208]",Alice,-1.053
7,0.198,0.200,"[0.0, 0.0, 1.0, 0.0, 0.0]",no,0.002,0.010,"[0.196, 0.191, 0.198, 0.207, 0.21]",Alice,1.470
8,0.229,0.267,"[0.0, 0.0, 0.0, 1.0, 0.0]",yes,0.037,0.139,"[0.196, 0.191, 0.198, 0.207, 0.21]",Alice,0.337
9,0.209,0.333,"[0.0, 0.0, 0.0, 0.0, 1.0]",no,0.124,0.373,"[0.195, 0.19, 0.197, 0.208, 0.209]",Alice,-2.078


It seems to update too much during the update steps (step 0-5). Using the example database of [1, 2, 3, 4, 5] and a workload of two stacked 5x5 identity matrices, during the first 5 queries consisting of the first identity matrix, the database updates the first few vectors way too much. 

In [83]:
def pmw2(workload, x, analyst_labels, T, eps=0.01, k=0, 
         show_messages=True, to_return='pd', show_plot=False, show_failure_step=True):
    """
    Implement Private Multiplicative Weights Mechanism (PMW) on a workload of
    linear queries. 

    Algorithm Parameters: 
    - workload = workload of queries (M x k numpy array)
    - x = true database (M x 1 numpy array)
    - T = update threshold
    - eps = privacy budget
    - k = number of update steps PER ANALYST
    - analyst_labels = list of analyst names corresponding to each query in the workload
    
    Output Controls: 
    - show_messages argument determines whether the function will print information such as 
    error scale, threshold, update steps used, etc.
    - to_return argument determines what the function will return. 
        - if 'pd', pmw() returns pandas df with test data for each 
        query in the workload(showing query, d_t_hat, updated, algo_ans, real_ans, 
        abs_error, rel_error). 
        - if 'update_count', pmw() returns the update count for the total
        amount of queries.
    - show_plot - T/F whether the function will display a plot
    - show_failure_step - T/F whether function prints what step failure mode is reached
    """ 
    
    update_steps = {}
    for analyst in list(set(analyst_labels)): 
        update_steps[analyst] = k # each analyst starts with k update steps
    
    # initialize constants
    m = x.size  # database len
    n = x.sum()
    eta = (math.log(m, np.e) ** (1 / 4)) / (math.sqrt(n))
    delta = 1 / (n * math.log(n, np.e))
    x_norm = x / np.sum(x)
    
    # initialize synthetic databases at time 0 (prior to any queries)
    x_t = np.ones(m) / m
    y_t = np.ones(m) / m

    # initialize tracker lists to construct pandas dataframe at the end 
    x_list = [x_t] # create a list of x_t synthetic database at every time step
    update_list = []
    update_count = 0
    pmw_answers = []
    update_times = [] # record times that database is updated
    d_t_hat_list = []
    
    def lazy_round():
        """
        "Lazy Round" of querying using the stored synthetic database, x_t, in list x_list.
        
        We call this the lazy round because it is contrasted with the updated step where we update the 
        sythetic database and answer the query using the real database.
        """
        update_list.append('no')
        pmw_answers.append(np.dot(query, x_list[time]))
        x_list.append(x_list[time].round(3))
    
    # inititate first instance of SVT with half the budget and k updates; will be reset in the main loop
    SVTtrigger = False 
    SVTepsilon1 = ((eps/2)/2)
    SVTepsilon2 = ((eps/2)/2)
    rho = np.random.laplace(loc=0, scale=(1/SVTepsilon1), size=1)[0]
    
    for time, query in enumerate(workload):
        
        analyst = analyst_labels[time]
        
        # Do one round of sparse vector technique 
        
        # Compute noisy answer by adding Laplacian noise
        a_t = np.random.laplace(loc=0, scale=(2*k/SVTepsilon2), size=1)[0]
        a_t_hat = (np.dot(query, x_norm)*n ) + a_t

        # Difference between noisy and maintained histogram answer
        d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
        
        # Lazy round: use synthetic base to answer the query
        if (abs(d_t_hat) <= T + rho):
            d_t_hat_list.append(d_t_hat)
            lazy_round()
            continue

        # update round: update histogram and return noisy answer
        else:
            #make a new noisy query answer using some of the leftover budget
            a_t = np.random.laplace(loc=0, scale=(2*k/eps), size=1)[0]
            a_t_hat = (np.dot(query, x_norm)*n ) + a_t
            d_t_hat = a_t_hat - (n*np.dot(query, x_list[time]))
            d_t_hat_list.append(d_t_hat)
            update_times.append(time)
            
            # step a
            if d_t_hat < 0:
                r_t = query
            else:
                r_t = np.ones(m) - query
            for i, v in enumerate(y_t):
                y_t[i] = x_list[time][i] * math.exp((d_t_hat/(2*n)) * query[i]) * 20 # 20 is the learning rate
            
            # step b
            x_t = y_t / np.sum(y_t)
            update_count = update_list.count('yes')
            
            # if threshold for num updates is reached, just do a lazy round (synthetic database) answer
            if update_steps[analyst] == 0: 
                if show_failure_step:
                    print(f'Failure mode reached at query number {time}: {query}')
                lazy_round()
                
            # if there are still update steps that the analyst can use, 
            # 1. update the synthetic database
            # 2. answer the query using the noisy answer from the database itself 
            else: 
                x_list.append(x_t.round(3))
                update_list.append('yes') # increment number of updates counter
                pmw_answers.append(a_t_hat / np.sum(x))
                update_steps[analyst] -= 1 # use one of analyst's update steps

    update_count = update_list.count('yes')      

    # calculate error
    real_ans = np.matmul(workload, x_norm)
    abs_error = np.abs(pmw_answers - real_ans)
    rel_error = np.abs(abs_error / np.where(real_ans == 0, 0.000001,
                                                real_ans))
    
    if show_messages:
        np.set_printoptions(suppress=True)
        """Print inputes/outputs to analyze each query"""
        print(f'Original database: {x}\n')
        print(f'Normalized database: {x_norm}\n')
        print(f'Updated Database = {x_t}\n')
        print(f'Update Count = {update_count}\n')
        print(f'{T=}\n')
        print(f'Error Scale Query Answer= {2*((2*k/eps)**2)}\n')
        print(f'Error Scale SVT= {2*((2*k/SVTepsilon2)**2)}\n')
        print(f'Update Parameter Scale = {eta}\n')
        print(f'{delta=}\n')
        
    if show_plot: 
        plt.title('Error across queries:')
        rel_line, = plt.plot(rel_error, label='Relative Error')
        abs_line, = plt.plot(abs_error, label='Absolute Error')
        for xc in update_times:
            plt.axvline(x=xc, color='red', label='Update Times', linestyle='dashed')
        plt.legend(handles=[abs_line,rel_line])
        plt.xticks(range(0, len(workload), round(len(workload)/5)))
    
    if to_return == "pd":
        # hacky fix: remove the first synthetic database to keep length of lists consistent with the
        # other lists that comprise of the pandas dataframe
        x_list.pop(0).tolist() 
        d = {
            'algo_ans': pmw_answers,
            'real_ans': real_ans.tolist(),
            'queries': workload.tolist(), 
            'updated': update_list,
            'abs_error': abs_error,               
            'rel_error': rel_error,
            'synthetic database': x_list,
            'analyst': analyst_labels,
            'd_t_hat': d_t_hat_list, 

             }
        test_data = pd.DataFrame(data=d)
        test_data = test_data.round(3)
        return test_data
    
    if to_return == "error":
        d = {'analyst': analyst_labels,
             'abs_error': abs_error,               
             'rel_error': rel_error,}
        data = pd.DataFrame(data=d)
        data = data.round(3)
        
        analyst_error = {}
        for analyst in list(set(analyst_labels)):
            analyst_error[analyst] = data[data.analyst==analyst]['abs_error'].sum()
        return analyst_error
pmw2(np.vstack((online_workloads.identity(5), online_workloads.identity(5))), x_example, ['Alice'] * 10, eps=1, T=5)


Original database: [1 2 3 4 5]

Normalized database: [0.06666667 0.13333333 0.2        0.26666667 0.33333333]

Updated Database = [0.2 0.2 0.2 0.2 0.2]

Update Count = 0

T=5

Error Scale Query Answer= 0.0

Error Scale SVT= 0.0

Update Parameter Scale = 0.29081910083756185

delta=0.024617958204590337



,algo_ans,real_ans,queries,updated,abs_error,rel_error,synthetic database,analyst,d_t_hat
0,0.2,0.067,"[1.0, 0.0, 0.0, 0.0, 0.0]",no,0.133,2.00,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,-2.0
1,0.2,0.133,"[0.0, 1.0, 0.0, 0.0, 0.0]",no,0.067,0.50,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,-1.0
2,0.2,0.200,"[0.0, 0.0, 1.0, 0.0, 0.0]",no,0.000,0.00,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,0.0
3,0.2,0.267,"[0.0, 0.0, 0.0, 1.0, 0.0]",no,0.067,0.25,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,1.0
4,0.2,0.333,"[0.0, 0.0, 0.0, 0.0, 1.0]",no,0.133,0.40,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,2.0
5,0.2,0.067,"[1.0, 0.0, 0.0, 0.0, 0.0]",no,0.133,2.00,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,-2.0
6,0.2,0.133,"[0.0, 1.0, 0.0, 0.0, 0.0]",no,0.067,0.50,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,-1.0
7,0.2,0.200,"[0.0, 0.0, 1.0, 0.0, 0.0]",no,0.000,0.00,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,0.0
8,0.2,0.267,"[0.0, 0.0, 0.0, 1.0, 0.0]",no,0.067,0.25,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,1.0
9,0.2,0.333,"[0.0, 0.0, 0.0, 0.0, 1.0]",no,0.133,0.40,"[0.2, 0.2, 0.2, 0.2, 0.2]",Alice,2.0


In [88]:
# pmw_independent: write pmw for one person. 
# create wrapper function called pmw_independent() that takes in the workloads and workload labels. Run PMW for each analyst, separate their workloads based on analysts. 

def pmw_independent(workload, x, analyst_labels, input_eps, input_threshold):

    indices = {} # k: analyst, v: row indices of queries in the workloads
    for i, analyst in enumerate(analyst_labels):
        if analyst not in indices.keys(): 
            indices[analyst] = []
        indices[analyst].append(i)

    workloads = {} # k: analyst, v: the analyst's workload
    for analyst in indices.keys():
        workloads[analyst] = w[indices['Alice'], :]

    for analyst in workloads.keys():
        print(i)
        pmw_split(workload=workloads[analyst], x=database, T=5, analyst_labels=[analyst]*len(workloads[analyst]), eps=input_eps, T=input_threshold)
        
pmw_independent(np.vstack((online_workloads.identity(5), online_workloads.identity(5))), x_example, ['Alice'] * 10, input_eps=1, input_threshold=5)
    

SyntaxError: keyword argument repeated (<ipython-input-88-ceaafa4e6304>, line 18)

In [7]:
# pmw_naive
# write version of PMW where analysts can run out of privacy budget if they use too much of others' budgets
# 

PMW independent

Initialize an instance of PMW for each analyst with alpha =T and their share of the privacy budget
Analysts don’t share anything - PB or Synthetic database
Each analyst is only allowed to query their instance of PMW

Naive PMW

Initialize an instance of PMW with alpha = T and the whole privacy budget 
All analysts sharing everything - privacy budget and synthetic database
Allow every analyst to query that instance of PMW

Split PMW

Initialize a single instance of PMW for each analyst with alpha = T and the entire privacy budget
Split the update steps proportionally to each analyst based on their share of the privacy budget
There exists cases where some analysts have more privacy budgets than others - i.e. alice owns 50 percent of the data
The difference between split and PMW - is that in Split, everyone shares a synthetic database
Inference steps are infamously non-monotonic
Allow any analyst to answer from the PMW instance and only allow them to cause an update step if they own any unused update steps
